# CH.2 Patterns for Cleaner Python


# 2.1 assert

用來確保程式 **內部** 不能有特定錯誤。
```python
# assert syntax
# expression2 是錯誤訊息
assert_stmt ::= "assert" expression1 ["," expression2]
```

python 直譯器在執行階段會轉為以下程式碼，如果加入加入 -O 或是 -OO 來最佳化編譯  
`__debug__` 為 False，assert 就會被忽略，因此別用 assert 當作正式上線的抓 error 方法
```python
if __debug__:
    if not expression1:
        raise AssertionError(expression2)
```

以下是一些關於 assert 的陷阱
+ Checking for admin privileges with an assert statement is dangerous. (因為編譯後會消失)
+ Asserts That Never Fail (如果讓第一個 statement 為 tuple，那將永遠成立)

In [2]:
counter = 10
assert (
    counter == 9,
    'It should have counted all the items'
)

<>:2: SyntaxWarning: assertion is always true, perhaps remove parentheses?
<>:2: SyntaxWarning: assertion is always true, perhaps remove parentheses?
/tmp/ipykernel_545534/927924879.py:2: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  assert (


**Key Takeaways**
+ Python’s assert statement is a debugging aid that tests a condition as an internal self-check in your program.
+ Asserts should only be used to help developers identify bugs.
They’re not a mechanism for handling run-time errors.
+ Asserts can be globally disabled with an interpreter setting

# 2.2 Complacent Comma Placement

如果你有一個包含人名的 list，逗號要如以下這樣放置
+ 在添加元素時才容易看出改了什麼
+ 有些 source control system 是 line-based ，只改一行可能偵測不到
+ 確保有輸入逗號
```python
names = [
    'Alice',
    'Bob',
    'Dilbert'
]
```

如果忘記打逗號 將會變成 `['Alice', 'BobDilbert']`

In [3]:
names = [
    'Alice',
    'Bob' # <- Missing comma!
    'Dilbert'
]
print(names)

['Alice', 'BobDilbert']


**Key Takeaways**
+ Smart formatting and comma placement can make your list,
dict, or set constants **easier to maintain.**
+ Python’s string literal concatenation feature can work to your
benefit, or introduce hard-to-catch bugs.

# 2.3 Context Managers and the with


with 能幫助我們寫出乾淨且可讀性更高的程式，是一種 context manager，能在特定區塊擁有資源，離開後就會銷毀  
\
我們能重新改寫程式碼
```python
f = open('hello.txt', 'w')
try:
    f.write('hello, world')
finally:
    f.close()
```
現在簡潔多了
```python
with open('a.txt','r') as f:
    f.write('123')
```

with 也能拿來可以用來做 threading lock
+ [about threading lock](https://ithelp.ithome.com.tw/articles/10254439)

In [4]:
import threading
some_lock = threading.Lock()

# 離開後自動 lock.release()
with some_lock:
    # do something
    pass

## Supporting with in Your Own Objects

如果要自己實作和 with 互動的物件，需遵守 context managers (一種 protocol or interface)  
實作出 `__enter__` 和 `__exit__` 

In [5]:
class myClass:
    def __init__(self,name):
        self.name = name
    
    def __enter__(self):
        self.file = open(self.name,'w')
        return self.file
    
    # exc_type , exc_value , trackback is needed
    def __exit__(self , exc_type , exc_value , trackback):
        if self.file:
            self.file.close()
            
with myClass('a.txt') as f:
    # do something
    pass

## generator-based 的寫法 (generator 在 6.5 會介紹)
使用 contextlib 提供的 decorator

In [6]:
from contextlib import contextmanager

@contextmanager
def managed_file(filename):
    try:
        f = open(filename,'w')
        yield f
    finally:
        f.close()

with managed_file('hello.txt') as f:
    f.write('hello, world!')
    f.write('bye now')

## Writing Pretty APIs With Context Managers
context manager 是很彈性的，如果善用就能定義出好用的 APIs  
  
以下程式碼能製造出縮排效果

In [7]:
class Indenter:
    def __init__(self):
        self.level = 0
    def __enter__(self):
        self.level += 1
        return self
    def __exit__(self, exc_type, exc_val, exc_tb):
        self.level -= 1
    def print(self, text):
        print('*' * self.level + text)

In [8]:
with Indenter() as indent:
    indent.print('hi!')
    with indent:
        indent.print('hello')
        with indent:
            indent.print('bonjour')
    indent.print('hey')

*hi!
**hello
***bonjour
*hey


**Key Takeaways**
+ The with statement simplifies exception handling by encapsulating standard uses of try/finally statements in so-called
**context managers**.
+ Most commonly it is used to manage the **safe acquisition and
release of system resources**. Resources are acquired by the
with statement and released automatically when execution
leaves the with context.
+ Using with effectively can help you avoid resource leaks and
make your code easier to read.


# 2.4 Underscores, Dunders, and More

## 前單底線
一種提示，告訴別人這是私有成員，但python沒有明確的 public/private 成員之分，還是能夠存取的到。  

但如果寫一個檔案叫做 `my_module.py`
```python
# my_module.py

def external_func():
    return 23

def _internal_func():
    return 42
```
前單底線的 function 會被忽略
```python
>>> from my_module import * 
>>> _internal_func()
"name '_internal_func' is not defined"
```
直接引用 my_module 還是能使用

```python
>>> import my_module
>>> my_module._internal_func()
42
```

## 前雙底線 Double Leading Underscore: “__var”
+ 在編譯的時候，會幫他自動改名
+ 用來避免 private member 被 overwrite
+ 在 class 內部的存取名稱不變，在 class 外才會受影響

In [9]:
# __hi 被重新命名成  _a__hi

class a:
    def __init__(self):    
        return 
    def __hi():
        print('hi')

i = a()
print(dir(i))

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_a__hi']


In [11]:
class Test:
    def __init__(self):
        self.foo = 11
        self._bar = 23
        self.__baz = 23

class ExtendedTest(Test):
    def __init__(self):
        super().__init__()
        self.foo = 'overridden'
        self._bar = 'overridden'
        self.__baz = 'overridden'

e = ExtendedTest()

print(e._ExtendedTest__baz) 

overridden


這樣將會出錯，因為 __baz 被改名為 _ExtendedTest__baz
```python
>>> print(e.__baz) 
>>> AttributeError: 'ExtendedTest' object has no attribute '__baz'
```

底下是一個很神奇的情況，`__mangled` 被 python 直譯器更改命名，竟然跟全域變數相連了

In [13]:
_MangledGlobal__mangled = 23

class MangledGlobal:
    def test(self):
        return __mangled

MangledGlobal().test()

23

## 後單底線  Single Trailing Underscore: “var_”
用來躲保留字
```python
class_ = 10 
```

## 前後雙底線 Double Leading and Trailing Underscore:“__var__”
+ 又叫做 `dunders`，是 `double underscore` 的縮寫，所以 `__init__` 會被叫做 `dunders init`
+ 通常都是作為 python 核心 function，不建議這樣命名，因為未來可能跟 python 更新有衝突

## 單底線 Single Underscore: “_”
用來當作小廢物變數

**Key Takeaways**
+ Single Leading Underscore “_var”: Naming convention indicating a name is meant for internal use. Generally not enforced by the Python interpreter (except in wildcard imports)
and meant as a hint to the programmer only.
+ Single Trailing Underscore “var_”: Used by convention to
avoid naming conflicts with Python keywords.
+ Double Leading Underscore “__var”: Triggers name mangling when used in a class context. Enforced by the Python interpreter.
+ Double Leading and Trailing Underscore “__var__”: Indicates special methods defined by the Python language. Avoid
this naming scheme for your own attributes.
+ Single Underscore “_”: Sometimes used as a name for temporary or insignificant variables (“don’t care”). Also, it represents the result of the last expression in a Python REPL session.

# 2.5 A Shocking Truth About String Formatting
以下將介紹 4 種格式化字串的方法

## #1 – “Old Style” String Formatting
跟 c 語言很相似

In [ ]:
name = 'bob'
print('Hello, %s' % name)

errno = 500
print('%x' % errno)

print('Hey %(name)s, there is a 0x%(errno)x error!' % {"name": name, "errno": errno })

## #2 – “New Style” String Formatting

In [ ]:
name = 'bob'
print('Hello, {}'.format(name))

errno = 500
print('Hey {name}, there is a 0x{errno:x} error!'.format(name=name, errno=errno))

## #3 – Literal String Interpolation (Python 3.6+)


我們寫了 greet function，用來回傳 f-string
```python
def greet(name, question):
    return f"Hello, {name}! How's it {question}?"
```
greet 經過 disassemble 後，會發現被轉成類似底下這種格式
```python
def greet(name, question):
    return ("Hello, " + name + "! How's it " + question + "?")
```
但實際上 f-string 會更快，因為使用到了 `BUILD_STRING` 來做最佳化

In [ ]:
import dis

def greet(name, question):
    return f"Hello, {name}! How's it {question}?"

dis.dis(greet)

附上 f-string formatter
| sign | mean | 
| -------- | -------- |
| :5     | 寬度 5    |
| :5.5     | 寬5小數點5    |
| : < > ^     | 左、右、置中   
| : * #      | 不夠的用 * # replace|

## #4 – Template Strings

沒有格式化規範，當需要處裡由使用者輸入的字串時可用，因功能少複雜度低

In [ ]:
from string import Template

t = Template('hi! $name')
my_name = 'eric'
t.substitute(name = my_name)

但如果有人想盜取機密資料，他能透過 #2 format string 存取到 __globals__ 內的資料

In [ ]:
SECRET = 'this-is-a-secret'

class Error:
    def __init__(self):
        pass

err = Error()

user_input = '{error.__init__.__globals__[SECRET]}'
user_input.format(error=err)

但如果用 template，就能避免此問題，template string 會去檢查輸入合不合法

In [ ]:
user_input = '${error.__init__.__globals__[SECRET]}'

# 可以執行看看 會出現錯誤
# Template(user_input).substitute(error=err)

## 我們該使用哪種 format string
**Dan’s Python String Formatting Rule of Thumb:** \
\
If your format strings are **user-supplied**, use Template 
Strings to avoid security issues.  \
\
Otherwise, use Literal  String Interpolation if you’re on Python 3.6+, and “New 
Style” String Formatting if you’re not. 

**Key Takeaways**
+ Perhaps surprisingly, there’s more than one way to handle
string formatting in Python.
+ Each method has its individual pros and cons. Your use case
will influence which method you should use.
+ If you’re having trouble deciding which string formatting
method to use, try my String Formatting Rule of Thumb.


## 2.6 指派運算子(3.8+)
這個酷東西能在指派的同時還有return value在指派的同時還有return value

In [1]:
a = '123'

# n 先指派為 len(a)
# 回傳出 n 給 if 判斷式
if (n := len(a)) > 10:
    print(f"List is too long ({n} elements, expected <= 10)")

In [16]:
with open('input.txt','w') as f:
  f.write(f"1\n2\n3\n")

with open('input.txt','r') as f:
  while line := f.readline():
    print(line)

1

2

3



## 2.7 “The Zen of Python” Easter Egg

In [17]:
import this

The Zen of Python, by Tim Peters

Beautiful is better than ugly.
Explicit is better than implicit.
Simple is better than complex.
Complex is better than complicated.
Flat is better than nested.
Sparse is better than dense.
Readability counts.
Special cases aren't special enough to break the rules.
Although practicality beats purity.
Errors should never pass silently.
Unless explicitly silenced.
In the face of ambiguity, refuse the temptation to guess.
There should be one-- and preferably only one --obvious way to do it.
Although that way may not be obvious at first unless you're Dutch.
Now is better than never.
Although never is often better than *right* now.
If the implementation is hard to explain, it's a bad idea.
If the implementation is easy to explain, it may be a good idea.
Namespaces are one honking great idea -- let's do more of those!


In [18]:
# 在 python shell 打這行會有彩蛋
import antigravity